In [1]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


In [ ]:
%cd yolov5
%pip install -qr requirements.txt

data yaml file 구성

In [1]:
import yaml
with open('./datas/data.yaml','w') as f:
    yaml.dump({
        'names':['Short sleeve top','Long sleeve top','Short sleeve outwear', 'Long sleeve outwear','Vest','Sling','Shorts','Trousers','Skirt','Short sleeve dress','Long sleeve dress','Vest dress','Sling dress'],
        'nc':13,
        'train':'./datas/train/image',
        'val':'./datas/validation/image/'
    },f)
with open('./datas/data.yaml','r') as f:
    print(yaml.load(f,Loader=yaml.FullLoader))

{'names': ['Short sleeve top', 'Long sleeve top', 'Short sleeve outwear', 'Long sleeve outwear', 'Vest', 'Sling', 'Shorts', 'Trousers', 'Skirt', 'Short sleeve dress', 'Long sleeve dress', 'Vest dress', 'Sling dress'], 'nc': 13, 'train': './datas/train/image', 'val': './datas/validation/image/'}


Dataset label txt file 생성

In [1]:
import glob
train_anno_dir = './datas/train/annos/*'
train_imgs_dir = './datas/train/image/*'
train_save_dir = './datas/train/labels/'
valid_anno_dir = './datas/validation/annos/*'
valid_imgs_dir = './datas/validation/image/*'
valid_save_dir = './datas/validation/labels/'
train_annos = glob.glob(train_anno_dir)
train_images = glob.glob(train_imgs_dir)
valid_annos = glob.glob(valid_anno_dir)
valid_images = glob.glob(valid_imgs_dir)

In [4]:
import json
from PIL import Image
from tqdm import tqdm
def to_yolo_coord(org_coord,width,height):
    x1,y1,x2,y2 = org_coord
    w= x2-x1
    h = y2-y1
    return [str(x1/width),str(y1/height),str(w/width),str(h/height)]

In [44]:
print(train_images[0])
print(train_annos[0])

./datas/train/image\000001.jpg
./datas/train/annos\000001.json


In [45]:
for i in tqdm(range(len(train_annos))):
    cur_img = Image.open(train_images[i])
    w,h = cur_img.size
    with open(train_annos[i],'r') as f:
        json_ver = json.loads(f.read())
        res = []
        for item_idx in range(1,5):
            try:
                temp_coord = json_ver[f'item{item_idx}']['bounding_box']
                yolo_coord = to_yolo_coord(temp_coord,w,h)
                category = json_ver[f'item{item_idx}']['category_id']
                bbox_string = ' '.join(yolo_coord)
                res.append(f'{category} {bbox_string}')
            except:
                break
        with open(train_save_dir+str(i+1).zfill(6)+'.txt','w',encoding="utf-8") as txt_f:
            txt_f.write('\n'.join(res))

100%|██████████| 191961/191961 [1:52:41<00:00, 28.39it/s]  


In [46]:
for i in tqdm(range(len(valid_annos))):
    cur_img = Image.open(valid_images[i])
    w,h = cur_img.size
    with open(valid_annos[i],'r') as f:
        json_ver = json.loads(f.read())
        res = []
        for item_idx in range(1,5):
            try:
                temp_coord = json_ver[f'item{item_idx}']['bounding_box']
                yolo_coord = to_yolo_coord(temp_coord,w,h)
                category = json_ver[f'item{item_idx}']['category_id']
                bbox_string = ' '.join(yolo_coord)
                res.append(f'{category} {bbox_string}')
            except:
                break
        with open(valid_save_dir+str(i+1).zfill(6)+'.txt','w',encoding="utf-8") as txt_f:
            txt_f.write('\n'.join(res))

100%|██████████| 32153/32153 [17:36<00:00, 30.44it/s]


In [3]:
train_save_dir = glob.glob('./datas/train/labels/*')
train_imgs_dir = './datas/train/image/'
# len(train_save_dir)

In [6]:
for i in tqdm(range(len(train_save_dir))):
    with open(train_save_dir[i],'r') as f:
        with open(train_imgs_dir+str(i+1).zfill(6)+'.txt','w',encoding="utf-8") as txt_f:
            txt_f.write(f.read())

100%|██████████| 191961/191961 [14:38<00:00, 218.54it/s]


데이터 반만 쓰기

In [1]:
import glob
from os import remove
from PIL import Image
from tqdm import tqdm

train_imgs_dir = './datas/train/image/*.jpg'
train_txt_dir = './datas/train/image/*.txt'
train_remain_dir = './datas/train/half/'
valid_imgs_dir = './datas/validation/image/*.jpg'
valid_txt_dir = './datas/validation/image/*.txt'
valid_remain_dir = './datas/validation/half/'
train_imgs = glob.glob(train_imgs_dir)
train_txts = glob.glob(train_txt_dir)
valid_imgs = glob.glob(valid_imgs_dir)
valid_txts = glob.glob(valid_txt_dir)

In [2]:
for i in tqdm(range(len(train_imgs)-1,len(train_imgs)//2,-1)):
    cur_img = Image.open(train_imgs[i])
    cur_img.save(train_remain_dir+str(i).zfill(6)+'.jpg')
    remove(train_imgs[i])
    with open(train_txts[i],'r') as f:
        with open(train_remain_dir+str(i).zfill(6)+'.txt','w') as txt_f:
            txt_f.write(f.read())
    remove(train_txts[i])

100%|██████████| 47990/47990 [26:19<00:00, 30.39it/s]


In [8]:
for i in tqdm(range(len(valid_imgs)-1,len(valid_imgs)//2,-1)):
    cur_img = Image.open(valid_imgs[i])
    cur_img.save(valid_remain_dir+str(i).zfill(6)+'.jpg')
    remove(valid_imgs[i])
    with open(valid_txts[i],'r') as f:
        with open(valid_remain_dir+str(i).zfill(6)+'.txt','w') as txt_f:
            txt_f.write(f.read())
    remove(valid_txts[i])

100%|██████████| 16076/16076 [08:10<00:00, 32.74it/s]
